<a href="https://colab.research.google.com/github/gpricechristie/hsstWebServiceDemo/blob/main/restfulWebService.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RESTful database backed web service
When the database and website functions are made available for other applications to use its often called a web service. REpresentational State Transfer (REST) a popular architecture using HTTP methods/verbs (GET, POST, PUT, PATCH, DELETE) to access service methods.

As for the previous example,this code uses flask to create an SQLite database backed website thats rendered through the use of html templates. In this example we also include some additional functions that allow the database to be quried and written to by other applications.

As in previous examples we also use pyngrok to expose the locally hosted web service to the public internet


In [ ]:
!pip install pyngrok
from flask import Flask, render_template, request, redirect, g, jsonify
from pyngrok import ngrok
import sqlite3
import os

##Create the website database
Use the sqlite3 package to create a new database (need the parent directory to exist first), connect to it, and create the same database as used in the previous example

In [ ]:
if not os.path.exists('db'):
    os.makedirs('db')
conn = sqlite3.connect("db/test.db")

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS computing_data;")
cursor.execute("CREATE TABLE IF NOT EXISTS computing_data ( fname TEXT, lname TEXT, valueofcomputing TEXT );")

##Boilerplate code to build website
As for the StaticWebsite notebook we need to create the flask server object, and authenticate and create the ngrok tunnel

In [ ]:
app = Flask(__name__)

ngrok.set_auth_token("2OxbwWyt7aHTrLuvIkLgdvgmSLA_81zjJCaMLzNbzx89uDb8r")
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, "5000"))

##Functions to render the site
The set of functions below are the same as for the previous dynamic website example:


*   Open the databae at the start of each request
*   Close the database at the send of each request
*   Write to the database via the /add_data URL when the html webform is used 
*   Render the page, including reading the database to add the table content to the page

In [ ]:
@app.before_request
def before_request():
    g.db = sqlite3.connect("db/test.db")

@app.teardown_request
def teardown_request(exception):
    if hasattr(g, 'db'):
        g.db.close()

@app.route('/add_data', methods = ['POST'])
def add_data():
    fname = request.form['firstname']
    lname = request.form['lastname']
    valueof = request.form['valueofcomputing']

    g.db.execute("INSERT INTO computing_data(fname, lname, valueofcomputing) VALUES (?,?,?)", [fname,lname,valueof])
    g.db.commit()

    return redirect('/')

@app.route('/')
def write_dynamic_page():
    data=g.db.execute("SELECT * FROM computing_data").fetchall()
    displayComments=[]
    for entry in data:
        displayComments.append(entry[0]+" "+entry[1]+" thinks that clinical computing is "+entry[2])

    return render_template('webform.html',len = len(displayComments),displayComments=displayComments)

##Functions for the RESTful interface

A request to the service URL /get_all_comment_names using the HTTP verb GET calls the below function. This queries all the data from the website, formats it as a JSON dataset and returns this to the client application via HTTP 

In [ ]:
@app.route('/get_all_comment_names', methods=['GET'])
def get_all_comment_names():
    data = g.db.execute("SELECT fname,lname FROM computing_data").fetchall()
    return jsonify(data)

A request to the /add_comment URL using the GET verb adds data to the database. The function requires the additinal parameters to be included in the requesting URL using the '?, &' convention. The start of the parameter list is indicated with a queston mark '?', with each parameter then defined as <parameter name>=<value> and separated by an ampersand '&'. An example URL is thus: 

server:port/add_comment?fname=gareth&lname=Price&comment="totally+rubbish. 

The function returns the '204' HTTP code that indicates to the client that the request was sucessful.

In [ ]:
@app.route('/add_comment', methods=['GET'])
def add_comment():
    query_parameters = request.args

    fname = query_parameters.get('fname')
    lname = query_parameters.get('lname')
    comment = query_parameters.get('comment')

    g.db.execute("INSERT INTO computing_data(fname, lname, valueofcomputing) VALUES (?,?,?)", [fname, lname, comment])
    g.db.commit()

    return ('', 204)

##Run application
As previously also need to load the html template and images to local file system. 

In [ ]:
!mkdir templates
!mkdir static
!wget https://raw.githubusercontent.com/gpricechristie/hsstWebServiceDemo/main/templates/webform.html -P ./templates
!wget https://raw.githubusercontent.com/gpricechristie/hsstWebServiceDemo/main/static/christie.jpg -P ./static

Run the application to create the local webserver. Click on the ngrok.io address from above to visit the page you have just created.

In [ ]:
if __name__ == "__main__":
        app.run()

#Demonstrate the RESTfulweb service interface
To interact with the RESTful service first open the site via the ngrok URL and add some data to it via the web form.

Open the command line window on your local computer (ie the laptop you are working on):


1.   In the windows menu type 'cmd'

##Get content of the web service database via /get_all_comment_names method

To use the command line you will need the cURL command or an equivalent. This comes as standard in Windows 10 or above.  

1.   At the command prompt type "curl \<ngrok URL\>/get_all_comment_names" where \<ngrok URL\> is the https ngrok address from above.
2.   This should then return the list of the names you have entered into the web form
3.   If you do not have the curl tool you can enter the same URL (including get_all_comment_names address) into a web browser. This should then display the list of names

##Add additional data to the database via the /add_comment method

Again using the cURL command in the cmd window

1.   At the comand prompt type curl "\<ngok URL\>/add_comment?fname=\<first name\>&lname=\<last name\>&comment=\<your choice of comment\>" --insecure (be sure to include the quotation marks around the URL you pass to cURL)
2.   Here \<ngrok URL\> is the ngrok URL from above
3.   \<first name\>, \<last name\> and\<comment\> are your choice of first name, last name and comment respectively. When entering the comment you can use the '+' character in place of whitespaces.
4.   The --insecure argument to the cURL command is used to circumvent some of the https security that we would implement on a production website
5.   An example command would thus be

curl "https://\<ngrok URL>/add_comment?fname=John&lname=Wick&comment=would+prefer+more+action" --insecure

6.   Refresh the displayed web page - it should now render the additional data you have added to the site






